In [1]:
# Section 1 – Install & Import Dependencies
!pip install streamlit pyngrok bcrypt pyjwt pandas nltk textstat --quiet
# Install Hugging Face Transformers and Pytorch (or Tensorflow if preferred)
!pip install transformers torch sentencepiece rouge_score --quiet

# Download NLTK resources
import nltk
nltk.download('punkt')
from pyngrok import ngrok
ngrok.kill()

# Set up your ngrok authtoken (REPLACE this with your actual token!)
!ngrok config add-authtoken 34ShfKeTV8CoS2zyEOQ5ynOTeEe_68W9dTvLEEgEnbF5hQxdh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
%%writefile app1.py
import streamlit as st
import sqlite3
import bcrypt
import pandas as pd
import jwt
from datetime import datetime, timedelta
import numpy as np
import time
import re
import nltk
from textstat import textstat
import random
import torch
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer, PegasusForConditionalGeneration, PegasusTokenizer, T5ForConditionalGeneration, T5Tokenizer
from rouge_score import rouge_scorer
import math
import pandas as pd

import plotly.express as px
# --- Global Configurations ---
SECRET_KEY = "your-long-and-very-secret-key-for-jwt-goes-here"
MOCK_OTP_DURATION_MINUTES = 5

# --- NLTK Downloads (ensure they run once) ---
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

# --- 1. Page Configuration & Custom CSS (Light Colors/Better UI) ---
st.set_page_config(
    page_title="LLM AI",
    layout="wide",
)

# New CSS for a softer, lighter design and specific Module 2 features
custom_css = """
<style>
    /* Global Container */
    .main .block-container {
        max-width: 1200px;
        padding-top: 2rem;
    }
    /* Header/Title Bar */
    h1 {
        background: linear-gradient(135deg, rgba(13, 109, 253, 0.8), rgba(147, 112, 219, 0.8), rgba(255, 105, 180, 0.8), rgba(238, 130, 238, 0.8));
        color: white; /* Changed text color to white for better contrast */
        padding: 1.5rem; /* Increased padding */
        border-radius: 15px; /* More rounded corners */
        text-align: center; /* Centered text */
        font-size: 30px; /* Larger font size */
        margin-bottom: 2rem !important; /* Increased margin */
        border-left: none; /* Removed left border */
        backdrop-filter: blur(10px); /* Glassy effect */
        -webkit-backdrop-filter: blur(10px); /* Safari support */
        box-shadow: 0 8px 20px rgba(0, 0, 0, 0.15); /* More prominent shadow */
        border: 1px solid rgba(255, 255, 255, 0.3); /* Light border */
    }
    /* General Card/Form Container */
    div[data-testid="stTabs"], .score-card, .key-features, .upload-area, .output-card {
        border: 1px solid rgba(201, 210, 242, 0.5); /* Adjusted border color with transparency */
        border-radius: 15px; /* More rounded corners */
        padding: 2rem; /* Increased padding */
        box-shadow: 0 6px 18px rgba(0, 0, 0, 0.08); /* Adjusted shadow */
        background-color: rgba(255, 255, 255, 0.6); /* Semi-transparent white background */
        backdrop-filter: blur(5px); /* Subtle glassy effect */
        -webkit-backdrop-filter: blur(5px); /* Safari support */
    }
    /* Button Style (Primary Blue) */
    .stButton>button {
        width: 100%;
        background: linear-gradient(45deg, #0d6efd, #6610f2); /* Gradient button */
        color: white;
        border-radius: 8px; /* More rounded buttons */
        border: none;
        margin: 5px 0;
        transition: all 0.3s ease; /* Smooth transition */
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    }
    .stButton>button:hover {
        background: linear-gradient(45deg, #0b5ed7, #550acc); /* Darker gradient on hover */
        box-shadow: 0 6px 12px rgba(0, 0, 0, 0.2);
    }
     /* OTP Success/Error */
    div[data-testid="stSuccess"], div[data-testid="stError"] {
        border-radius: 8px;
    }
    /* Readability Scores/Progress Bars (Improved Visuals) */
    .score-card {
        padding: 1.5rem; /* Increased padding */
        background-color: rgba(247, 249, 255, 0.7); /* Semi-transparent light background */
        margin-bottom: 15px; /* Increased margin */
        border-radius: 12px; /* Rounded corners */
        backdrop-filter: blur(4px); /* Subtle glassy effect */
        -webkit-backdrop-filter: blur(4px); /* Safari support */
        box-shadow: 0 2px 6px rgba(0, 0, 0, 0.05);
    }
    .score-value { font-size: 2.5em; font-weight: bold; color: #1e3a8a; }
    .score-label { font-size: 1em; color: #4b5563; margin-top: 5px; }

    /* Complexity Chart Colors (Matching visual cues) */
    .stBarChart { opacity: 0.9; }

    /* New: Specific style for the mock code warning */
    .mock-code-warning {
        background-color: #ffe0b2; /* Light orange/amber */
        color: #ff9800; /* Darker orange text */
        padding: 10px;
        border-radius: 8px;
        border-left: 5px solid #ff9800;
        margin-top: 10px;
        font-weight: bold;
        font-size: 1.1em;
    }

     /* Input fields */
    .stTextInput>div>div>input, .stTextArea>div>div>textarea {
        background-color: rgba(255, 255, 255, 0.7);
        border-radius: 8px;
        border: 1px solid #c9d2f2;
        padding: 10px;
    }

    /* Select boxes */
    .stSelectbox>div>div>div>div {
         background-color: rgba(255, 255, 255, 0.7);
         border-radius: 8px;
         border: 1px solid #c9d2f2;
         padding: 5px 10px;
    }

    /* Progress bar colors for readability */
    .progress-bar-container {
        width: 100%;
        background-color: #e0e0e0;
        border-radius: 5px;
        margin-top: 10px;
        overflow: hidden; /* Ensures the inner bar stays within bounds */
    }

    .progress-bar {
        height: 8px;
        border-radius: 5px;
        transition: width 0.5s ease-in-out;
    }

    /* Green for easy (high Flesch Ease, low grade indices) */
    .fe-green .progress-bar, .fk-green .progress-bar, .gf-green .progress-bar, .smog-green .progress-bar {
        background-color: #28a745;
    }

    /* Yellow/Orange for intermediate */
    .fe-yellow .progress-bar, .fk-yellow .progress-bar, .gf-yellow .progress-bar, .smog-yellow .progress-bar {
        background-color: #ffc107;
    }

    /* Red for difficult (low Flesch Ease, high grade indices) */
    .fe-red .progress-bar, .fk-red .progress-bar, .gf-red .progress-bar, .smog-red .progress-bar {
        background-color: #dc3545;
    }

    /* Radar chart customization (handled by Plotly, but ensuring container styling doesn't interfere) */
    .stPlotlyChart {
        background-color: rgba(255, 255, 255, 0.6); /* Match card background */
        border-radius: 12px;
        padding: 15px;
        box-shadow: 0 2px 6px rgba(0, 0, 0, 0.05);
         backdrop-filter: blur(4px);
        -webkit-backdrop-filter: blur(4px);
    }


</style>
"""
st.markdown(custom_css, unsafe_allow_html=True)

# =============================================================================
#  2. DATABASE AND AUTHENTICATION BACKEND FUNCTIONS
# =============================================================================

def init_db():
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()

    # 1. Create table with all columns (for new DBs)
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            email TEXT PRIMARY KEY,
            username TEXT,
            password_hash BLOB NOT NULL,
            role TEXT NOT NULL
        )
    ''')

    # 2. Database Migration Check (Ensuring 'username' column exists)
    try:
        # Check if the 'username' column exists and is populated
        c.execute("SELECT username FROM users LIMIT 1")
    except sqlite3.OperationalError:
        # If the column doesn't exist (old schema), add it
        c.execute("ALTER TABLE users ADD COLUMN username TEXT")

    # 3. Update existing users (like the default admin) to have a username
    c.execute("UPDATE users SET username = 'AdminUser' WHERE email='admin@ai' AND username IS NULL")

    # 4. Add default admin user if not exists (using the correct, complete column list)
    c.execute("SELECT * FROM users WHERE email='admin@ai'")
    if not c.fetchone():
        admin_email, admin_pass, admin_username = "admin@ai", "Infosys", "AdminUser"
        hashed_pass = bcrypt.hashpw(admin_pass.encode(), bcrypt.gensalt())
        c.execute("INSERT INTO users (email, username, password_hash, role) VALUES (?, ?, ?, ?)", (admin_email, admin_username, hashed_pass, "Admin"))

    # OTP table
    c.execute('''
        CREATE TABLE IF NOT EXISTS password_resets (
            email TEXT PRIMARY KEY,
            otp TEXT NOT NULL,
            expiry_time TIMESTAMP NOT NULL
        )
    ''')

    conn.commit()
    conn.close()

def get_all_users():
    conn = sqlite3.connect('llm_users.db')
    df = pd.read_sql_query("SELECT email, username, role FROM users", conn)
    conn.close()
    return df

def delete_user(email):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("DELETE FROM users WHERE email=?", (email,))
    conn.commit()
    conn.close()

def user_exists(email):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("SELECT email FROM users WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()
    return result is not None

def update_password(email, new_password):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    hashed_password = bcrypt.hashpw(new_password.encode('utf-8'), bcrypt.gensalt())
    c.execute("UPDATE users SET password_hash = ? WHERE email = ?", (hashed_password, email))
    # Clear the OTP record after successful reset
    c.execute("DELETE FROM password_resets WHERE email=?", (email,))
    conn.commit()
    conn.close()
    return "Password updated successfully!"

def verify_password(plain_password, hashed_password):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed_password)

def register_user(email, username, password, role="General User"):
    if len(password) < 6:
        return "Password must be at least 6 characters long."
    if not username.strip():
        return "Username cannot be empty."

    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()

    # Check if email already exists.
    c.execute("SELECT * FROM users WHERE email=?", (email,))
    if c.fetchone(): conn.close(); return "Email already exists."

    # Check if username already exists.
    c.execute("SELECT * FROM users WHERE username=?", (username,))
    if c.fetchone(): conn.close(); return "Username already exists."

    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    c.execute("INSERT INTO users (email, username, password_hash, role) VALUES (?, ?, ?, ?)", (email, username, hashed_password, role))
    conn.commit()
    conn.close()
    return "User registered successfully! Please log in."

def generate_token(email, role):
    payload = {'exp': datetime.utcnow() + timedelta(hours=1), 'iat': datetime.utcnow(), 'sub': email, 'role': role}
    return jwt.encode(payload, SECRET_KEY, algorithm='HS256')

def decode_token(token):
    try: return jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
    except: return None

def authenticate_user(email, password):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("SELECT password_hash, role FROM users WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()
    if result:
        hashed_password_from_db, role = result
        if verify_password(password, hashed_password_from_db):
            return generate_token(email, role)
    return None

# --- Mock OTP Functions ---
def generate_and_store_otp(email):
    otp = str(random.randint(100000, 999999))
    expiry_time = datetime.utcnow() + timedelta(minutes=MOCK_OTP_DURATION_MINUTES)

    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("REPLACE INTO password_resets (email, otp, expiry_time) VALUES (?, ?, ?)", (email, otp, expiry_time))
    conn.commit()
    conn.close()
    return otp

def verify_otp(email, otp):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("SELECT otp, expiry_time FROM password_resets WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()

    if result:
        stored_otp, expiry_time_str = result
        try:
            expiry_time = datetime.fromisoformat(expiry_time_str)
        except ValueError:
            expiry_time = datetime.strptime(expiry_time_str, '%Y-%m-%d %H:%M:%S.%f')

        if stored_otp == otp and datetime.utcnow() < expiry_time:
            return True
    return False


# =============================================================================
#  3. NLP Model Management (Mock/Load)
# =============================================================================

@st.cache_resource
def load_summarization_model(model_name):
    st.info(f"Loading {model_name}...")
    try:
        # Load small/base models for faster response in a demo setting
        if model_name == "BART":
            model = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)
        elif model_name == "Pegasus":
            model = pipeline("summarization", model="google/pegasus-xsum", device=0 if torch.cuda.is_available() else -1)
        elif model_name == "FLAN-T5":
            model = pipeline("summarization", model="google/flan-t5-small", device=0 if torch.cuda.is_available() else -1)
        else:
            return None
        st.success(f"{model_name} loaded.")
        return model
    except Exception as e:
        st.error(f"Error loading {model_name}: {e}. Ensure you have installed the models correctly.")
        return None

@st.cache_resource
def load_paraphrasing_model(model_name):
    st.info(f"Loading {model_name} for paraphrasing...")
    try:
        if model_name == "FLAN-T5 (Paraphrase)":
            model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
            tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
        elif model_name == "BART (Paraphrase)":
            model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
        else:
            return None, None
        st.success(f"{model_name} loaded.")
        return model, tokenizer
    except Exception as e:
        st.error(f"Error loading {model_name}: {e}. Ensure you have installed the models correctly.")
        return None, None

# =============================================================================
#  4. READABILITY ANALYSIS FUNCTIONS
# =============================================================================


# =============================================================================
def calculate_readability_scores(text):
    """Calculates multiple readability scores for the given text."""
    if not text.strip(): return None, None, None, None
    clean_text = re.sub(r'[^a-zA-Z0-9\s.,;\'"!?()]', '', text)
    if not clean_text.strip(): return None, None, None, None
    try:
        flesch_kincaid = textstat.flesch_kincaid_grade(clean_text)
        gunning_fog = textstat.gunning_fog(clean_text)
        smog_index = textstat.smog_index(clean_text)
        flesch_reading_ease = textstat.flesch_reading_ease(clean_text)
    except Exception as e:
        return None, None, None, None
    return flesch_kincaid, gunning_fog, smog_index, flesch_reading_ease

def get_score_color_class(score, index_type='flesch_kincaid'):
    """Returns a CSS class based on the score range for visual feedback."""
    if score is None: return ""
    if index_type == 'flesch_kincaid':
        if score <= 5: return "fk-green"    # Very Easy
        elif score <= 10: return "fk-yellow" # Intermediate
        else: return "fk-red"               # Complex
    elif index_type == 'gunning_fog':
        if score <= 8: return "gf-green"
        elif score <= 12: return "gf-yellow"
        else: return "gf-red"
    elif index_type == 'smog_index':
        if score <= 8: return "smog-green"
        elif score <= 12: return "smog-yellow"
        else: return "smog-red"
    elif index_type == 'flesch_ease':
        if score >= 90: return "fe-green"    # Very Easy
        elif score >= 60: return "fe-yellow" # Standard
        else: return "fe-red"                # Difficult
    return ""

def display_score_card(label, score, max_val, index_type):
    """Displays an enhanced, color-coded score card."""
    score_display = f"{score:.1f}" if score is not None else "N/A"

    if index_type == 'flesch_ease':
        max_val_progress = 100
        progress_percent = (score / max_val_progress * 100) if score is not None else 0
    else:
        max_val_progress = 20
        if score is not None:
            capped_score = min(score, max_val_progress)
            # Invert progress bar for grade levels: 0 score (easiest) = 100% progress
            progress_percent = (1 - (capped_score / max_val_progress)) * 100
        else:
            progress_percent = 0

    color_class = get_score_color_class(score, index_type)

    st.markdown(f"""
    <div class="score-card {color_class}">
        <div class="score-value">{score_display}</div>
        <div class="score-label">{label}</div>
        <div class="progress-bar-container">
            <div class="progress-bar" style="width: {progress_percent:.1f}%;"></div>
        </div>
    </div>
    """, unsafe_allow_html=True)


# =============================================================================
#  5. UI SECTION: READABILITY ANALYSIS (Updated with Plotly Radar Chart)
# =============================================================================
def readability_analysis_ui():
    """Renders the professional readability analysis dashboard UI."""
    st.markdown("<h1>📈 Document Readability Dashboard</h1>", unsafe_allow_html=True)
    st.markdown("Analyze the complexity and grade level of your text using industry-standard metrics.", unsafe_allow_html=True)

    # --- Input Area ---
    st.markdown("""<div class="upload-area"><h3>Text Input for Analysis</h3></div>""", unsafe_allow_html=True)

    uploaded_file = st.file_uploader("Upload a text document (.txt) or a simple PDF/DOCX for extraction attempt.", type=["txt", "pdf", "docx"], label_visibility="collapsed", key="readability_file")

    col_text, col_button = st.columns([6, 1])
    with col_text:
        text_input = st.text_area("Paste your content here", height=200, help="Enter text directly for full readability analysis.", key="readability_text_input")
    with col_button:
        st.markdown("<div style='height: 155px;'></div>", unsafe_allow_html=True)
        analyze_button = st.button("Analyze Content", key="run_readability_analysis", use_container_width=True)

    content_to_analyze = ""
    run_analysis_flag = False

    # Logic to process input and trigger analysis (kept simple for focus)
    if analyze_button or uploaded_file is not None:
        if analyze_button and text_input:
            content_to_analyze = text_input
            run_analysis_flag = True
        elif uploaded_file is not None:
            try:
                if uploaded_file.type == "text/plain":
                    content_to_analyze = uploaded_file.read().decode("utf-8")
                    run_analysis_flag = True
                else:
                    st.warning("Using sample text for complexity demonstration.")
                    content_to_analyze = "The complex nexus of computational and theoretical frameworks necessitates a multi-modal approach to extant problematics, which are often non-linear and require significant, interdisciplinary expertise."
                    run_analysis_flag = True
            except Exception as e:
                st.error(f"Error reading file: {e}")
                run_analysis_flag = False
        elif analyze_button and not text_input:
             st.warning("Please paste text or upload a file to analyze.")

    # Store analyzed content in session state for persistence
    if run_analysis_flag:
        st.session_state.analyzed_content = content_to_analyze
        st.session_state.readability_run = True
    elif not analyze_button and uploaded_file is None:
        if 'readability_run' in st.session_state:
             st.session_state.readability_run = False
        if 'analyzed_content' in st.session_state:
            st.session_state.analyzed_content = ""

    # --- Output Area ---
    if st.session_state.get('readability_run') and st.session_state.get('analyzed_content'):
        content_to_analyze = st.session_state.analyzed_content
        fk_grade, gunning_fog, smog_index, flesch_ease = calculate_readability_scores(content_to_analyze)
        word_count = len(content_to_analyze.split())

        # --- Score Cards Section ---
        st.subheader("Key Readability Metrics 🎯")
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            display_score_card("F-K Grade Level", fk_grade, 18, 'flesch_kincaid')
        with col2:
            display_score_card("Gunning Fog Index", gunning_fog, 20, 'gunning_fog')
        with col3:
            display_score_card("SMOG Grade Index", smog_index, 20, 'smog_index')
        with col4:
            display_score_card("Flesch Reading Ease", flesch_ease, 100, 'flesch_ease')

        st.markdown("---")

        # --- Complexity Level & RADAR Chart ---
        st.subheader("Readability Profile Visualization (Radar Chart) 📊")

        if fk_grade is not None and flesch_ease is not None:

            # Determine complexity level for the header
            if fk_grade <= 5: level = "Elementary School (K-5)"; color = '#28a745'
            elif fk_grade <= 8: level = "Middle School (6-8)"; color = '#ffc107'
            elif fk_grade <= 12: level = "High School (9-12)"; color = '#fd7e14'
            else: level = "College / Post-Graduate"; color = '#dc3545'

            st.markdown(f"""
                <div style='text-align:center; padding: 15px; background-color:{color}; color:white; border-radius:10px; margin-bottom: 20px;'>
                    <h4>Estimated Audience Level: <b>{level}</b></h4>
                    <p>Flesch-Kincaid Grade: {fk_grade:.1f} | Flesch Reading Ease: {flesch_ease:.1f}</p>
                </div>
            """, unsafe_allow_html=True)


            # --- Plotly Radar Chart Implementation ---

            # 1. Normalize scores to a 0-100 scale for a unified radar chart.
            # We want the HIGHER score (closer to the edge) to mean EASIER text.

            # Grade-level indices (FK, GF, SMOG): Lower is easier. Max cap at 15 for visualization.
            # Normalized Score = 100 - (Actual Score / 15) * 100
            fk_norm = max(0, min(100, 100 - (fk_grade / 15) * 100))
            gf_norm = max(0, min(100, 100 - (gunning_fog / 15) * 100))
            smog_norm = max(0, min(100, 100 - (smog_index / 15) * 100))

            # Flesch Reading Ease (FE): Higher is easier. Max cap at 100.
            # Normalized Score = Actual Score
            fe_norm = max(0, min(100, flesch_ease))

            # Create a DataFrame for Plotly (must be long format)
            chart_data = pd.DataFrame(dict(
                r=[fe_norm, fk_norm, gf_norm, smog_norm],
                theta=['Flesch Ease (100=Easy)', 'F-K Grade (100=Easy)', 'Gunning Fog (100=Easy)', 'SMOG Index (100=Easy)'],
                actual_score=[f'{flesch_ease:.1f}', f'{fk_grade:.1f}', f'{gunning_fog:.1f}', f'{smog_index:.1f}']
            ))

            fig = px.line_polar(chart_data,
                                r='r',
                                theta='theta',
                                line_close=True,
                                color_discrete_sequence=['#4c78a8'] # Single color line for a profile plot
                               )

            # Customizing the Plotly Radar Chart
            fig.update_traces(fill='toself', name='Your Text')
            fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        range=[0, 100],
                        tickvals=[0, 25, 50, 75, 100],
                        ticktext=['Hard (0)', 'Difficult (25)', 'Standard (50)', 'Easy (75)', 'Very Easy (100)'],
                        showline=True,
                        linecolor='gray'
                    )),
                showlegend=False,
                height=500
            )

            # Add hover text for actual scores
            fig.update_traces(hovertemplate='<b>%{theta}</b><br>Normalized Ease: %{r:.1f}<br>Actual Score: %{customdata} <extra></extra>',
                              customdata=chart_data['actual_score'])

            st.plotly_chart(fig, use_container_width=True)

            st.info("The **Readability Profile** (Radar Chart) visualizes your text across four metrics on a unified 0-100 'Ease' scale. A score of **100** (outer edge) indicates maximum ease, and **0** (center) indicates maximum difficulty.")
        else:
            st.info("The text provided is too short or not parsable for full complexity analysis.")

        st.markdown("---")
        st.subheader("Document Metrics Summary")

        col_count1, col_count2, col_count3 = st.columns(3)
        with col_count1:
            st.metric("Total Words", word_count)
        with col_count2:
            sentence_count = textstat.sentence_count(content_to_analyze)
            st.metric("Total Sentences", sentence_count)
        with col_count3:
            syllable_count = textstat.syllable_count(content_to_analyze)
            st.metric("Total Syllables", syllable_count)

        st.markdown("""
            <div class="key-features">
                <p><b>Key Features:</b> ⚡ Real-time scoring, 👁️ Visual complexity indicators (color-coded cards & radar chart), 📚 Comprehensive text metrics.</p>
            </div>
            """, unsafe_allow_html=True)
    else:
        st.info("Upload a document or paste text above and click 'Analyze Content' to generate the professional readability dashboard.")
# =============================================================================
#  6. UI SECTION: MODULE 2 - Summarizer & Paraphraser (Full)
# =============================================================================
def summarizer_paraphraser_ui():
    st.markdown("<h1>📚 Text Processing</h1>", unsafe_allow_html=True)

    # --- Text input with an explicit 'Send' button ---
    st.markdown("---")
    col_nlp_text, col_nlp_button = st.columns([6, 1])

    default_text = st.session_state.get('last_nlp_input', "Paste your text here for summarization/paraphrasing...")

    with col_nlp_text:
        text_input = st.text_area("Paste your text here for summarization/paraphrasing:", height=200, value=default_text, key="nlp_input_area")
    with col_nlp_button:
        st.markdown("<div style='height: 155px;'></div>", unsafe_allow_html=True) # Spacer
        process_button = st.button("Send", key="run_nlp_process", use_container_width=True)

    text_to_process = ""

    # Logic to trigger processing on button click
    if process_button:
        if text_input and text_input.strip() != "Paste your text here for summarization/paraphrasing...":
            st.session_state.original_text = text_input
            st.session_state.last_nlp_input = text_input
            st.session_state.summary_processed = False
            st.session_state.paraphrase_processed = False
            # Calculate original readability for the paraphrasing tab
            fk_grade, _, _, _ = calculate_readability_scores(text_input)
            st.session_state.original_fk_grade = fk_grade
        else:
            st.warning("Please paste some text before clicking 'Send'.")
            return

    text_to_process = st.session_state.get('original_text')
    if not text_to_process:
        st.info("Paste text above and click 'Send' to enable summarization and paraphrasing tools.")
        return

    # --- UPDATED TABS: Added History ---
    sum_tab_main, sum_tab_history = st.tabs(["Summary Generator", "History"])

    with sum_tab_main:
        st.subheader("Multi-level Summarization")
        col_model, col_length = st.columns([1.5, 2])
        with col_model:
            sum_model_name = st.selectbox("Select Model", ["BART", "Pegasus", "FLAN-T5"], key="sum_model")
        with col_length:
            summary_length = st.radio("Summary Length", ["Short", "Medium", "Long"], horizontal=True, key="sum_length")

        num_words = len(text_to_process.split())
        min_len = max(10, int(num_words * 0.05))
        max_len = min(200, int(num_words * 0.5))

        if summary_length == "Short":
            min_len = max(min_len, 20)
            max_len = min(max_len, 80)
        elif summary_length == "Medium":
            min_len = max(min_len, 40)
            max_len = min(max_len, 120)
        else:
            min_len = max(min_len, 80)
            max_len = min(max_len, 200)

        if max_len < min_len + 10:
             max_len = min_len + 10

        if st.button(f"Generate {summary_length} Summary", key="run_summarize"):
            with st.spinner(f"Generating summary using {sum_model_name}..."):
                summarizer = load_summarization_model(sum_model_name)
                if summarizer:
                    summary_output = summarizer(text_to_process, max_length=max_len, min_length=min_len, do_sample=False)[0]['summary_text']

                    r1 = random.uniform(0.35, 0.45)
                    rL = random.uniform(0.25, 0.35)

                    st.session_state.summary_output = summary_output
                    st.session_state.rouge_scores = {'r1': r1, 'rL': rL}
                    st.session_state.summary_processed = True
                else:
                    st.error("Model loading failed.")

        if st.session_state.get('summary_processed') and st.session_state.get('summary_output'):
            st.markdown("---")
            st.subheader("Comparison: Original vs. Summarized Text")

            col_orig, col_sum = st.columns(2)
            with col_orig:
                st.markdown("**Original Text** (Words: {word_count})".format(word_count=len(st.session_state.original_text.split())))
                st.info(st.session_state.original_text)
            with col_sum:
                st.markdown("**Summarized Text** (Words: {word_count})".format(word_count=len(st.session_state.summary_output.split())))
                st.success(st.session_state.summary_output)

            # --- NEW: Word Count Comparison Graph ---
            st.markdown("---")
            st.subheader("Word Count Comparison")
            word_count_orig = len(st.session_state.original_text.split())
            word_count_sum = len(st.session_state.summary_output.split())

            chart_data = pd.DataFrame({
                'Text Type': ['Original Text', 'Summarized Text'],
                'Word Count': [word_count_orig, word_count_sum]
            }).set_index('Text Type')

            st.bar_chart(chart_data, use_container_width=True)


            st.markdown("---")
            st.subheader("Evaluation: Mock ROUGE Scores Visualized")

            r1_score = st.session_state.rouge_scores['r1']
            rL_score = st.session_state.rouge_scores['rL']

            rouge_data = pd.DataFrame({
                'Metric': ['ROUGE-1 F-Measure', 'ROUGE-L F-Measure'],
                'Score': [r1_score, rL_score],
            })

            rouge_data = rouge_data.set_index('Metric')
            st.bar_chart(rouge_data, use_container_width=True)

            st.json({
                "rouge1_fmeasure": f"{r1_score:.4f}",
                "rougeL_fmeasure": f"{rL_score:.4f}",
                "Note": "ROUGE scores are mocked; true evaluation requires a reference summary."
            })

    with sum_tab_history:
        st.subheader("Summary History")
        st.markdown("""
            <div style="padding: 10px; border: 1px dashed #ccc; border-radius: 8px;">
                <p>This section is designed to store and display your past summarization results.</p>
                <p><strong>Mock History Items:</strong></p>
                <ul>
                    <li>Summary 1 (BART, Short) - 2025-10-21: R-L 0.32</li>
                    <li>Summary 2 (Pegasus, Medium) - 2025-10-20: R-L 0.28</li>
                </ul>
            </div>
        """, unsafe_allow_html=True)

    # --- UPDATED TABS: Added History ---
    para_tab_main, para_tab_history = st.tabs(["Paraphrasing Tool", "History"])

    with para_tab_main:
        st.subheader("Complexity-based Paraphrasing")
        para_col1, para_col2 = st.columns(2)
        with para_col1:
            complexity_level = st.selectbox("Target Complexity Level", ["Simplified (Lower Reading Grade)", "More Complex (Higher Reading Grade)", "Maintain Complexity"], key="para_complexity")
        with para_col2:
            para_model_name = st.selectbox("Select Model", ["FLAN-T5 (Paraphrase)", "BART (Paraphrase)"], key="para_model")

        if st.button("Generate Paraphrase", key="run_paraphrase"):
            with st.spinner(f"Generating paraphrase using {para_model_name} for '{complexity_level}'..."):
                model, tokenizer = load_paraphrasing_model(para_model_name)
                if model and tokenizer:
                    if complexity_level == "Simplified (Lower Reading Grade)":
                        prompt = f"Paraphrase the following text to simplify the vocabulary and sentence structure: {text_to_process}"
                    elif complexity_level == "More Complex (Higher Reading Grade)":
                        prompt = f"Paraphrase the following text to use more sophisticated vocabulary and longer sentences: {text_to_process}"
                    else:
                        prompt = f"Paraphrase the following text: {text_to_process}"
                    input_ids = tokenizer.encode(prompt, return_tensors='pt', max_length=512, truncation=True)
                    outputs = model.generate(input_ids, max_length=int(len(text_to_process.split()) * 2.0), num_beams=5, early_stopping=True)
                    paraphrase_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

                    p_fk, _, _, _ = calculate_readability_scores(paraphrase_output)

                    st.session_state.paraphrase_output = paraphrase_output
                    st.session_state.paraphrase_fk_grade = p_fk
                    st.session_state.paraphrase_processed = True
                else:
                    st.error("Model loading failed.")

        if st.session_state.get('paraphrase_processed') and st.session_state.get('paraphrase_output'):
            st.markdown("---")
            st.subheader("Comparison: Original vs. Paraphrased")
            col_orig, col_para = st.columns(2)

            with col_orig:
                st.markdown("**Original Text** (F-K Grade: {fk:.1f})".format(fk=st.session_state.get('original_fk_grade', 0)))
                st.info(st.session_state.original_text)
            with col_para:
                st.markdown("**Paraphrased Text** (F-K Grade: {fk:.1f})".format(fk=st.session_state.paraphrase_fk_grade))
                st.success(st.session_state.paraphrase_output)

            # --- NEW: Readability Comparison Graph ---
            st.markdown("---")
            st.subheader("Readability Shift Visualization (Flesch-Kincaid)")

            original_fk = st.session_state.get('original_fk_grade', 10)
            paraphrase_fk = st.session_state.get('paraphrase_fk_grade', original_fk)

            # Cap grades at 18 for consistent charting scale (no longer needed to pass to st.line_chart)
            max_grade = 18

            chart_data = pd.DataFrame({
                'Stage': ['Original Text', 'Paraphrased Text'],
                'Flesch_Kincaid_Grade': [original_fk, paraphrase_fk]
            }).set_index('Stage')

            # FIX APPLIED: Removed unsupported arguments (y_label, y_max)
            st.line_chart(chart_data, use_container_width=True)
            st.markdown(f"**Y-Axis:** Flesch-Kincaid Grade (Max grade displayed: {max_grade})")

            if complexity_level == "Simplified (Lower Reading Grade)":
                expected = "Lower"
                actual_change = "Decreased" if paraphrase_fk < original_fk else "Increased/Maintained"
                st.info(f"Goal: **{expected}** the Flesch-Kincaid Grade. Actual change: {actual_change}. The graph visualizes this shift.")
            elif complexity_level == "More Complex (Higher Reading Grade)":
                expected = "Higher"
                actual_change = "Increased" if paraphrase_fk > original_fk else "Decreased/Maintained"
                st.info(f"Goal: **{expected}** the Flesch-Kincaid Grade. Actual change: {actual_change}. The graph visualizes this shift.")
            else:
                st.info("The graph visualizes the change in readability after paraphrasing.")

    with para_tab_history:
        st.subheader("Paraphrasing History")
        st.markdown("""
            <div style="padding: 10px; border: 1px dashed #ccc; border-radius: 8px;">
                <p>This section is designed to store and display your past paraphrasing results.</p>
                <p><strong>Mock History Items:</strong></p>
                <ul>
                    <li>Paraphrase 1 (Simplified) - 2025-10-21: FK Grade 8.5</li>
                    <li>Paraphrase 2 (Complex) - 2025-10-19: FK Grade 13.2</li>
                </ul>
            </div>
        """, unsafe_allow_html=True)


# =============================================================================
#  7. UI SECTION: CHAT UI (Full)
# =============================================================================
def real_time_chat_ui():
    st.markdown("<h1>💬 Real-Time Chat Assistant</h1>", unsafe_allow_html=True)
    st.info("This section provides real-time chat functionality, including the ability to upload files for context-aware conversation.")

    # Initialize states
    if 'messages' not in st.session_state: st.session_state.messages = [{'role': 'assistant', 'content': 'Hello! How can I help you today? You can upload a document for a context-aware chat.'}]
    if 'chat_history' not in st.session_state: st.session_state.chat_history = []

    with st.sidebar:
        st.header("Chat Controls")

        # New Chat Button
        if st.button("➕ New Chat", key="new_chat"):
            # Save current chat to history if it has more than just the greeting
            if len(st.session_state.messages) > 1: st.session_state.chat_history.append(st.session_state.messages)
            st.session_state.messages = [{'role': 'assistant', 'content': 'Hello! How can I help you today? You can upload a document for a context-aware chat.'}]
            st.session_state.chat_context = None
            st.rerun()

        # Delete All Chat History Button (New Feature)
        if st.button("🗑️ Delete All History", key="delete_history"):
            st.session_state.chat_history = []
            st.session_state.messages = [{'role': 'assistant', 'content': 'Chat history deleted. New session started.'}]
            st.session_state.chat_context = None
            st.rerun()

        st.subheader("Recent Chats")
        for i, chat in enumerate(st.session_state.chat_history):
            # Extract the first user prompt as the title
            user_prompt = next((msg['content'] for msg in chat if msg['role'] == 'user'), 'Chat')
            if st.button(f"📜 {user_prompt[:30]}...", key=f"chat_{i}"):
                st.session_state.messages = chat; st.rerun()

    # Display Messages
    for message in st.session_state.messages:
        with st.chat_message(message['role']): st.markdown(message['content'])

    # File Uploader for Context
    uploaded_context_file = st.file_uploader("Upload Document for Context (+)", type=["txt"], key="chat_file_upload")

    # Handle context loading
    if uploaded_context_file and 'chat_context' not in st.session_state:
        st.session_state.chat_context = uploaded_context_file.read().decode("utf-8")
        st.info("Document uploaded. Your next chat questions will use this document as context.")

    if 'chat_context' in st.session_state and st.session_state.chat_context:
        st.success(f"Context loaded: {len(st.session_state.chat_context.split())} words.")

    # Predefined Q&A (add more here as needed)
    predefined_qa = {
        "what is text summarization?": "Text summarization is the task of creating a shorter version of a text document or article while retaining the most important information.",
        "how does paraphrasing work?": "Paraphrasing involves rephrasing a text using different words and sentence structures while keeping the original meaning intact. It can be used to simplify complex text or make it more sophisticated.",
        "what are readability scores?": "Readability scores are metrics that estimate the difficulty a reader will have understanding a text. Examples include Flesch-Kincaid Grade Level, Gunning Fog Index, and Flesch Reading Ease.",
        "what are summarization and paraphrasing models?": "Summarization and paraphrasing models are types of large language models (LLMs) or natural language processing (NLP) models trained for specific text generation tasks. Summarization models create concise versions of texts, while paraphrasing models rephrase texts to alter complexity or style.",
        "hi": "Hello there! How can I assist you today?",
        "hello": "Hi! What can I do for you?"
    }

    # Mock Response Logic
    def get_bot_response(user_prompt, context=None):
        time.sleep(1) # Simulate thinking time

        # Check for predefined answers (case-insensitive and strip whitespace)
        cleaned_prompt = user_prompt.strip().lower()
        if cleaned_prompt in predefined_qa:
            return predefined_qa[cleaned_prompt]

        # Fallback to original mock response if no predefined answer
        if context:
            return f"**Context-Aware Mock Response:** Based on your document ({len(context.split())} words), I interpret your question about '{user_prompt[:20]}...' as requiring a complex answer. [Replace with actual LLM call and context window management]"
        else:
            return f"**Real-Time Mock Response:** This is a general response to: '{user_prompt[:20]}...'. [Replace with actual LLM call]"

    # Chat Input
    if prompt := st.chat_input("Ask me anything..."):
        st.session_state.messages.append({'role': 'user', 'content': prompt})
        with st.chat_message('user'): st.markdown(prompt)

        with st.chat_message('assistant'):
            with st.spinner("Thinking..."):
                context = st.session_state.get('chat_context')
                response = get_bot_response(prompt, context)
                st.markdown(response)
        st.session_state.messages.append({'role': 'assistant', 'content': response})

# =============================================================================
#  8. FORGOT PASSWORD UI (UPDATED WITH PERSISTENT MOCK OTP)
# =============================================================================
def forgot_password_ui():
    st.subheader("Forgot Password (Mock OTP)")

    if 'forgot_stage' not in st.session_state: st.session_state.forgot_stage = 0
    if 'mock_otp_code' not in st.session_state: st.session_state.mock_otp_code = None

    if st.session_state.forgot_stage == 0:
        with st.form("forgot_password_email_form", clear_on_submit=True):
            st.markdown("##### Step 1: Request Code")
            forgot_email = st.text_input("Enter your Email", key="forgot_email_0")

            if st.form_submit_button("Send Mock Reset Code"):
                if not user_exists(forgot_email):
                    st.error("Email not found. Please check your email address.")
                    st.session_state.mock_otp_code = None # Clear code on failure
                else:
                    otp_code = generate_and_store_otp(forgot_email)
                    st.session_state.forgot_email = forgot_email
                    st.session_state.forgot_stage = 1
                    # Store the OTP in session state to persist the warning
                    st.session_state.mock_otp_code = otp_code
                    st.success(f"A mock reset code has been 'sent' to {forgot_email}. It expires in {MOCK_OTP_DURATION_MINUTES} minutes.")
                    st.rerun() # Rerun to display Step 2 form

    if st.session_state.get('forgot_stage') == 1:
        # Display persistent mock code warning at the top of the form
        if st.session_state.mock_otp_code:
             st.markdown(f"""
                <div class="mock-code-warning">
                    🚨 **Mock Code (Copy this): {st.session_state.mock_otp_code}** </div>
            """, unsafe_allow_html=True)

        st.markdown(f"##### Step 2: Enter Code and New Password for {st.session_state.forgot_email}")
        with st.form("forgot_password_reset_form", clear_on_submit=True):
            user_otp = st.text_input("Enter Reset Code", key="user_otp")
            new_pass = st.text_input("Enter New Password (Min 6 chars)", type="password", key="forgot_new_pass_1") # Password length note added
            confirm_new_pass = st.text_input("Confirm New Password", type="password", key="forgot_confirm_pass_1")

            if st.form_submit_button("Reset Password"):
                if new_pass != confirm_new_pass:
                    st.error("New passwords do not match.")
                elif len(new_pass) < 6:
                    st.error("Password must be at least 6 characters long.")
                elif not verify_otp(st.session_state.forgot_email, user_otp):
                    st.error("Invalid or expired reset code. Please request a new one.")
                else:
                    # Success Path
                    message = update_password(st.session_state.forgot_email, new_pass)
                    st.success(message + " You can now log in.")
                    st.session_state.forgot_stage = 0
                    del st.session_state.forgot_email
                    del st.session_state.mock_otp_code # Clear mock code on success
                    time.sleep(2)
                    st.rerun()

        # Allow user to go back to request a new code
        if st.button("Go back to request new code"):
            st.session_state.forgot_stage = 0
            if 'forgot_email' in st.session_state: del st.session_state.forgot_email
            if 'mock_otp_code' in st.session_state: del st.session_state.mock_otp_code
            st.rerun()

# =============================================================================
#  9. UI SECTION: ADMIN DASHBOARD (NEW FEATURE)
# =============================================================================
def admin_dashboard_ui():
    st.markdown("<h1>👑 Admin Dashboard: User Management</h1>", unsafe_allow_html=True)

    tab_view, tab_add = st.tabs(["View & Delete Users", "Add New User"])

    with tab_view:
        st.subheader("Current Users")

        user_data = get_all_users()

        if not user_data.empty:
            st.dataframe(user_data, use_container_width=True, hide_index=True)

            st.markdown("---")
            st.subheader("Delete User")

            # Filter out the current admin from deletion list to prevent lockout
            users_to_delete = user_data[user_data['email'] != st.session_state.user_email]['email'].tolist()

            if not users_to_delete:
                st.warning("Only the current Admin user exists. Cannot delete the only Admin.")
                return

            email_to_delete = st.selectbox("Select User Email to Delete", users_to_delete, key="delete_email_select")

            if st.button(f"Permanently Delete User: {email_to_delete}", key="run_delete_user"):
                if email_to_delete == st.session_state.user_email:
                    st.error("You cannot delete your own active Admin account.")
                else:
                    delete_user(email_to_delete)
                    st.success(f"User '{email_to_delete}' has been deleted.")
                    st.rerun() # Refresh list

        else:
            st.info("No users found in the database.")

    with tab_add:
        st.subheader("Register New User")
        with st.form("admin_register_form"):
            new_email = st.text_input("Email", key="admin_reg_email")
            new_username = st.text_input("Username", key="admin_reg_username") # NEW FIELD
            new_password = st.text_input("New Password (Min 6 chars)", type="password", key="admin_reg_pass")
            confirm_password = st.text_input("Confirm Password", type="password", key="admin_reg_confirm")
            role = st.selectbox("Role", ["General User", "Admin"], key="admin_reg_role_select")

            if st.form_submit_button("Register New User"):
                if new_password != confirm_password:
                    st.error("Passwords do not match.")
                elif len(new_password) < 6: # New Validation
                    st.error("Password must be at least 6 characters long.")
                elif not new_username.strip():
                    st.error("Username cannot be empty.")
                else:
                    # Pass username to the registration function
                    message = register_user(new_email, new_username, new_password, role) # PASS USERNAME
                    if "successfully" in message:
                        st.success(message + " New user added.")
                        st.rerun()
                    else:
                        st.warning(message)

# =============================================================================
#  10. MAIN AUTHENTICATION ROUTER
# =============================================================================

# Initialize session states
if 'token' not in st.session_state: st.session_state.token = None
if 'current_view' not in st.session_state: st.session_state.current_view = 'Readability Analysis'
if 'forgot_stage' not in st.session_state: st.session_state.forgot_stage = 0
if 'messages' not in st.session_state: st.session_state.messages = [{'role': 'assistant', 'content': 'Hello! How can I help you today?'}]
if 'chat_history' not in st.session_state: st.session_state.chat_history = []
if 'theme' not in st.session_state: st.session_state.theme = 'Light'
if 'summary_processed' not in st.session_state: st.session_state.summary_processed = False
if 'paraphrase_processed' not in st.session_state: st.session_state.paraphrase_processed = False
if 'mock_otp_code' not in st.session_state: st.session_state.mock_otp_code = None
if 'last_nlp_input' not in st.session_state: st.session_state.last_nlp_input = ""
if 'readability_run' not in st.session_state: st.session_state.readability_run = False
if 'original_fk_grade' not in st.session_state: st.session_state.original_fk_grade = None
if 'paraphrase_fk_grade' not in st.session_state: st.session_state.paraphrase_fk_grade = None
if 'show_forgot_password' not in st.session_state: st.session_state.show_forgot_password = False


init_db()
payload = decode_token(st.session_state.token)

if payload is None:
    st.session_state.token = None
    st.session_state.forgot_stage = st.session_state.get('forgot_stage', 0)

    # Ensure all processing flags are reset on logout/no login
    st.session_state.summary_processed = False
    st.session_state.paraphrase_processed = False
    st.session_state.readability_run = False

    _, main_content, _ = st.columns([1, 1.5, 1])
    with main_content:
        st.markdown("<h1>➡ User Authentication</h1>", unsafe_allow_html=True)
        login_tab, register_tab = st.tabs(["Login", "Register"]) # Removed forgot_pass_tab

        with login_tab:
            if not st.session_state.get('show_forgot_password', False):
                with st.form("login_form"):
                    email = st.text_input("Email", placeholder="Enter your Email")
                    password = st.text_input("Password", type="password", placeholder="*****")
                    if st.form_submit_button("Sign In"):
                        token = authenticate_user(email, password)
                        if token:
                            st.session_state.token = token
                            st.session_state.messages = [{'role': 'assistant', 'content': 'Hello! How can I help you today?'}]
                            st.session_state.chat_history = []
                            st.session_state.forgot_stage = 0 # Reset stage on login
                            st.session_state.show_forgot_password = False # Reset on successful login
                            if 'mock_otp_code' in st.session_state: del st.session_state.mock_otp_code # Clear code on successful login
                            st.rerun()
                        else:
                            st.error("Invalid email or password.")

                # Added Forgot Password button here outside the form
                if st.button("Forgot Password?"):
                    st.session_state.forgot_stage = 0 # Reset stage to 0 to show email input
                    st.session_state.show_forgot_password = True
                    st.rerun() # Rerun to switch form

            if st.session_state.get('show_forgot_password', False):
                forgot_password_ui() # Call the forgot password UI function

        with register_tab:
            with st.form("register_form"):
                new_email = st.text_input("Email", key="reg_email")
                new_username = st.text_input("Username", key="reg_username") # NEW FIELD
                new_password = st.text_input("New Password (Min 6 chars)", type="password", key="reg_pass")
                confirm_password = st.text_input("Confirm Password", type="password", key="reg_confirm")
                role = st.selectbox("Role", ["General User", "Admin"], key="reg_role_select")
                if st.form_submit_button("Register"):
                    if new_password != confirm_password:
                        st.error("Passwords do not match.")
                    elif len(new_password) < 6:
                        st.error("Password must be at least 6 characters long.") # NEW VALIDATION
                    elif not new_username.strip():
                        st.error("Username cannot be empty.")
                    else:
                        message = register_user(new_email, new_username, new_password, role) # PASS USERNAME
                        if "successfully" in message: st.success(message)
                        else: st.warning(message)


else: # User is logged in
    role = payload['role']
    st.session_state.user_email = payload['sub']

    with st.sidebar:
        st.success(f"Logged in as: {st.session_state.user_email}")
        st.write(f"Your role is: **{role}**")
        st.markdown("---")

        st.subheader("Navigation")
        if role.lower() == "admin":
            if st.button("Admin Dashboard", key="nav_admin"):
                st.session_state.current_view = "Admin Dashboard"
            st.markdown("---") # Separator for admin section

        if st.button("Readability Analysis", key="nav_readability"):
            st.session_state.current_view = "Readability Analysis"
        if st.button("Text Processing", key="nav_text_processing"):
            st.session_state.current_view = "Text Processing"
        if st.button("Chat Assistant", key="nav_chat"):
             st.session_state.current_view = "Chat Assistant"


        st.markdown("---")
        if st.button("Logout"):
            st.session_state.token = None
            st.session_state.current_view = 'Readability Analysis'
            # Clear all session states relevant to user data/processing
            st.session_state.messages = []
            st.session_state.chat_history = []
            if 'chat_context' in st.session_state: del st.session_state.chat_context
            st.session_state.readability_run = False
            st.session_state.show_forgot_password = False # Ensure this is reset on logout
            st.rerun()

    # Render content based on role and selected view
    if role.lower() == "admin" and st.session_state.current_view == "Admin Dashboard":
        admin_dashboard_ui()
    elif st.session_state.current_view == "Readability Analysis":
        readability_analysis_ui()
    elif st.session_state.current_view == "Text Processing":
        summarizer_paraphraser_ui()
    elif st.session_state.current_view == "Chat Assistant":
        real_time_chat_ui()

Writing app1.py


In [ ]:
# Section 3 – Run Streamlit via Pyngrok
ngrok.kill() # Ensure all ngrok processes are stopped
public_url = ngrok.connect(8502)
print(f"🌍 Public URL: {public_url}")
!streamlit run app1.py --server.port 8502 --server.headless true

🌍 Public URL: NgrokTunnel: "https://steedless-wayne-unreproached.ngrok-free.dev" -> "http://localhost:8502"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.53.94.230:8502

2025-10-23 13:06:00.801252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761224760.963371     655 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761224761.034307     655 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761224761.076892     655 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same ta